# Analyze whole-brain activity
* Uses `ImagingTrialLoader` to load data

In [ ]:
from neuroprocessing.imagingtrials import ImagingTrialLoader
import matplotlib.pyplot as plt
import numpy as np
params = {
        "downsample_factor": 8,
        "aligner_target_num_features": 700,
        "secs_before_stim": 60, # only process frames starting at X seconds before stimulus
        "preprocess_prefix": "aligned_downsampled_",
        "process_prefix": 'processed_',
        "s3fs_toplvl_path": "/Users/ilya_arcadia/arcadia-neuroimaging-pruritogens/Videos",
        "local_toplvl_path": "/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/",
        "load_from_s3": False,
        "save_to_s3": False,
        'crop_px' : 20,
        'bottom_percentile' : 5
        }

trials = ImagingTrialLoader(params)
# exp_dates, trial_names = trials.filter_exp_and_trial_dirs(exp_dir="2024-03-20", 
#                                                           limb = "RHL",
#                                                         )
exp_dates, trial_names = trials.filter_exp_and_trial_dirs()

masks = trials.load_mask_files()
traces = trials.load_traces()
sync_infos = trials.get_sync_infos()
# Plot the traces

fig, ax = plt.subplots()
for trace, exp_date, trial_name, sync in zip(traces, exp_dates, trial_names, sync_infos):
    t = (np.arange(0, len(trace))) / (sync['framerate_hz'] / params['downsample_factor']) - params['secs_before_stim']
    trace = trace - trace[np.where(t >= 0)[0][0]]
    ax.plot(t, trace, label=exp_date + ' ' + trial_name)
# legend to the right of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.axvline(x=0, color='r', linestyle='--')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Response (a.u.)')
# plot masks
fig, axs = plt.subplots(ncols=len(masks), figsize=(15, 7))
for ax, mask, exp_date, trial_name in zip(axs, masks, exp_dates, trial_names):
    ax.imshow(mask, cmap='gray')
    # remove axis ticks
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()


# SVD and alignment to AI CCF

In [ ]:
from wfield import load_stack, approximate_svd, chunk_indices, SVDStack, load_allen_landmarks, allen_load_reference, allen_transform_regions, atlas_from_landmarks_file
from skimage import io
from neuroprocessing.scripts.analysis import _identify_trial_save_paths
import numpy as np
import os, sys

def _run_SVD(img_stack, n_SVD_components = 200):
    """ Wrapper for wfield's SVD function"""

    chunkidx = chunk_indices(len(img_stack),chunksize=256)
    _frame_averages = []
    for on,off in chunkidx:
        _frame_averages.append(img_stack[on:off].mean(axis=0))

    _frames_average = np.stack(_frame_averages).mean(axis = 0)
    U,SVT = approximate_svd(img_stack, _frames_average, 
                            k = n_SVD_components,
                            nframes_per_bin=10)
    return _frames_average, U,SVT

def reconstruct(u,svt,dims = None):
    if dims is None:
            dims = u.shape[:2]
    return (u@svt).reshape((*dims,-1)).transpose(-1,0,1).squeeze()


def run_SVD_and_alignment(exp_dir, trial_dir, params):
    trial_path, save_path = _identify_trial_save_paths(exp_dir, trial_dir, params)
    dat = load_stack(save_path / (params["preprocess_prefix"] + trial_dir + ".tif"))

    frames_average, U,SVT = _run_SVD(dat, n_SVD_components = 2)

    # reconstructed movie
    mov = reconstruct(U,SVT).reshape(dat.nframes,1,*U.shape[:2])
    mov = (mov*frames_average)+frames_average
    # save SVD
    np.save(save_path / ('U_' + params["process_prefix"] + trial_dir + '.npy'), U)
    np.save(save_path / ('SVT_' + params["process_prefix"] + trial_dir + '.npy'), SVT)
    np.save(save_path / ('frames_average_' + params["process_prefix"] + trial_dir + '.npy'), frames_average)
    io.imsave(save_path / ('svd_' + params["process_prefix"] + trial_dir + '.tif'), mov.squeeze().astype(np.uint16))

    from wfield.widgets import QApplication,RawViewer
    app_args = ['/Users/ilya_arcadia/miniconda3/envs/neuro/bin/wfield', 'open_raw', trial_path]
    app = QApplication(app_args)
    w = RawViewer(raw = dat,
                    mask = None,
                    folder = trial_path,
                    trial_onsets = None,
                    reference = 'dorsal_cortex')
    # close app
    app.exec_()
    
    del dat

    # AI alignment
    basedir = '/Users/ilya_arcadia/Neuroimaging_local/Processed/wfield_testing/Zyla_30min_RHL_40ugin10uL_1pt5pctISO_1_1/wfield_results'
    landmarks_json = os.path.join(trial_path,'dorsal_cortex_landmarks.json')
    U = np.load(os.path.join(basedir, 'U.npy'))
    SVT = np.load(os.path.join(basedir, 'SVT.npy'))

    stack = SVDStack(U,SVT)
    lmarks = load_allen_landmarks(os.path.join(basedir,'dorsal_cortex_landmarks.json'))

    ccf_regions_reference,proj,brain_outline = allen_load_reference('dorsal_cortex')
    # the reference is in allen CCF space and needs to be converted
    # this converts to warped image space (accounting for the transformation)
    ccf_regions = allen_transform_regions(None,ccf_regions_reference,
                                        resolution = lmarks['resolution'],
                                            bregma_offset = lmarks['bregma_offset'])
    atlas, areanames, brain_mask = atlas_from_landmarks_file(landmarks_json) # this loads the atlas in transformed coords

    # this does the transform (warps the original images)
    # stack.set_warped(1, M = lmarks['transform']) # this warps the spatial components in the stack
    stack.set_warped(False)
    




In [ ]:
from neuroprocessing.scripts.analysis import process_trial
params = {
        "downsample_factor": 8,
        "aligner_target_num_features": 700,
        "secs_before_stim": 60, # only process frames starting at X seconds before stimulus
        "preprocess_prefix": "aligned_downsampled_",
        "process_prefix": 'processed_',
        "s3fs_toplvl_path": "/Users/ilya_arcadia/arcadia-neuroimaging-pruritogens/Videos",
        "local_toplvl_path": '/Users/ilya_arcadia/Neuroimaging_local/Processed/wfield_testing', # "/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/",
        "load_from_s3": False,
        "save_to_s3": False,
        'crop_px' : 20,
        'bottom_percentile' : 5
        }
run_SVD_and_alignment('2024-03-20-F1Num8REP', 'Zyla_15min_RHL_salineInj_1pt5pctISO_1', params=params)

# Quick script to calculate FFT of signal

In [ ]:
## FFT of the ROI
import numpy as np
import matplotlib.pyplot as plt
# get roi at center of image
center = np.array(img.shape) // 2
roi = img[:,center[1]-50:center[1]+50, center[2]-50:center[2]+50]
roi_mean = np.mean(roi, axis=(1,2))
# calculate heart rate using FFT
from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq

fs = 10 # Hz
t_orig = np.arange(0, len(roi_mean)) / fs
roi_mean = roi_mean - np.mean(roi_mean)
N = len(roi_mean)
yf = fft(roi_mean)
yf = 2.0/N * np.abs(yf[0:N//2])

xf = fftfreq(N, 1/fs)[:N//2]
f, axs = plt.subplots(2, 1)
axs[0].plot(roi_mean)
axs[0].set_xlim(0, 200)
axs[1].plot(xf, yf)
plt.xlim(0.5,3)
plt.ylim(0, 3)

# find peaks
peaks, _ = find_peaks(yf, height=5, distance=10)
plt.plot(xf[peaks], yf[peaks], "x")